In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B")


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

C:\Users\spdlq\.conda\envs\level2-nlp-datacentric-nlp-04\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\spdlq\.cache\huggingface\hub\models--beomi--Llama-3-Open-Ko-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [5]:
# 텍스트 생성 예제
inputs = tokenizer("Hello, how are you?", return_tensors="pt")
outputs = model.generate(inputs.input_ids, max_length=50)
print(tokenizer.decode(outputs[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|>Hello, how are you? I'm AAA, the sales team leader of BBB.
Hello, how are you? I'm AAA, the sales team leader of BBB.
안녕하세요, 잘 지내셨나요? BBB사의 영업팀장


LangChain에서 Hugging Face 모델을 연결하기 위해, 먼저 Transformers 라이브러리로 모델을 로드한 뒤 pipeline 객체를 생성합니다.

In [21]:
from transformers import pipeline
from langchain import HuggingFacePipeline

# 파이프라인 생성
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1000,  # 생성할 최대 토큰 길이
    temperature=0.5,
    do_sample=True,  # 샘플링을 활성화하여 temperature를 사용  # 텍스트 생성의 다양성을 제어하는 하이퍼파라미터
    device=0
)

# LangChain의 HuggingFacePipeline 래퍼 사용
llama_chain = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
import time
# 프롬프트를 입력으로 모델에서 텍스트 생성
prompt = """
다음 텍스트에서 아스키코드 기반으로 노이즈가 들어간 단어들을 자연스럽게 고쳐줘:
"pI美대선I앞두고 R2fr단 발] $비해 감시 강화"
아스키코드기반 문자 대체를 통해 노이즈가 들어간 데이터야. 다시 디노이징 해줘"
"""

# 시간 측정 시작
start_time = time.time()
# 모델에서 텍스트 생성
result = llama_chain.invoke(prompt)
# 시간 측정 종료
end_time = time.time()

print(f"입력 프롬프트 : {prompt}")
# 결과 출력
print(f"출력 프롬프트 : {result}")

# 걸린 시간 출력
elapsed_time = end_time - start_time
print(f"걸린 시간: {elapsed_time:.2f}초")